# [Advent of Code 2019: Day 9](https://adventofcode.com/2019/day/9)

<h2>--- Day 9: Sensor Boost ---</h2>
<p>You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!</p>
<p>In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest <em>BOOST</em> program - Basic Operation Of System Test.</p>
<p>While BOOST (your puzzle input) is capable of boosting your sensors, for <span title="Oh sure, NOW safety is a priority.">tenuous safety reasons</span>, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a <em>complete Intcode computer</em>.</p>
<p><a href="https://adventofcode.com/2019/day/5">Your existing Intcode computer</a> is missing one key feature: it needs support for parameters in <em>relative mode</em>.</p>
<p>Parameters in mode <code>2</code>, <em>relative mode</em>, behave very similarly to parameters in <em>position mode</em>: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.</p>
<p>The important difference is that relative mode parameters don't count from address <code>0</code>. Instead, they count from a value called the <em>relative base</em>. The <em>relative base</em> starts at <code>0</code>.</p>
<p>The address a relative mode parameter refers to is itself <em>plus</em> the current <em>relative base</em>. When the relative base is <code>0</code>, relative mode parameters and position mode parameters with the same value refer to the same address.</p>
<p>For example, given a relative base of <code>50</code>, a relative mode parameter of <code>-7</code> refers to memory address <code>50 + -7 = <em>43</em></code>.</p>
<p>The relative base is modified with the <em>relative base offset</em> instruction:</p>
<ul>
    <li>Opcode <code>9</code> <em>adjusts the relative base</em> by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.</li>
</ul>
<p>For example, if the relative base is <code>2000</code>, then after the instruction <code>109,19</code>, the relative base would be <code>2019</code>. If the next instruction were <code>204,-34</code>, then the value at address <code>1985</code> would
    be output.</p>
<p>Your Intcode computer will also need a few other capabilities:</p>
<ul>
    <li>The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value <code>0</code> and can be read or written like any other memory. (It is invalid to try to access memory at a negative
        address, though.)</li>
    <li>The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.</li>
</ul>
<p>Here are some example programs that use these features:</p>
<ul>
    <li><code>109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99</code> takes no input and produces a <a href="https://en.wikipedia.org/wiki/Quine_(computing)">copy of itself</a> as output.</li>
    <li><code>1102,34915192,34915192,7,4,7,99,0</code> should output a 16-digit number.</li>
    <li><code>104,1125899906842624,99</code> should output the large number in the middle.</li>
</ul>
<p>The BOOST program will ask for a single input; run it in test mode by providing it the value <code>1</code>. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly,
    and finally output a BOOST keycode.</p>
<p>Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. <em>What BOOST keycode does it produce?</em></p>

In [1]:
import os
import unittest

class IntcodeComputer():
    
    OP_ADD = 1    
    OP_MULTIPLY = 2    
    OP_INPUT = 3    
    OP_OUTPUT = 4
    OP_JUMP_TRUE = 5
    OP_JUMP_FALSE = 6
    OP_LESS_THAN = 7
    OP_EQUALS = 8
    OP_MOD_REL = 9
    
    OP_HALT = 99
    
    PARAM_MODE_POS = '0'
    PARAM_MODE_IMD = '1'
    PARAM_MODE_REL = '2'
    
    NOUN_ADDR = 1
    VERB_ADDR = 2
    RESULT_ADDR = 0
    START_ADDR = 0
    
    INIT_VAL = 0
    
    def __init__(self, data = []):
        self.inputs = []
        self.memory = []
        self.initial_memory = []
        if data:
            self.load_memory(data)
        
    def load_memory(self, data):
        self.initial_memory = self.normalize_memory(data)
        self.reset()
        
    def expand_memory(self, addr):
        needed_mem = addr - (len(self.memory) - 1)
        if needed_mem > 0:
            self.memory += ([self.INIT_VAL] * needed_mem)
        else:
            raise Exception(f'Cannot expand memory for addr {addr}')
        
    def check_addr(self, addr):
        if addr < 0:
            raise Exception(f'Addr {addr}, cannot be negative')
        
        if addr >= len(self.memory):
            self.expand_memory(addr)
            
        return addr
            
        
    def reset(self):
        if self.memory:
            del self.memory[:]
        self.memory = self.initial_memory.copy()
        
        if self.inputs:
            del self.inputs[:]
        self.inputs = []
        
        self.output = None
        self.last_input = None
        
        self.instruction_ptr = self.START_ADDR
        self.relative_base = self.START_ADDR
        
    def add_input(self, data):
        self.inputs.append(data)
        
    def print_program(self):
        print("Program: {:02d}{:02d}".format(self.memory[self.NOUN_ADDR],self.memory[self.VERB_ADDR]))

    def normalize_memory(self, intcode):
        if type(intcode) is str:
            return list(map(int, intcode.split(',')))

        elif type(intcode) is list:
            if type(intcode[0]) is str:
                return list(map(int, intcode))
            else:
                return intcode

        else:
            raise Exception('Corrupt intcode')
            
    def get_paramater(self, mode):
        param = self.memory[self.instruction_ptr]
        self.instruction_ptr += 1
        
        if mode == self.PARAM_MODE_POS:
            addr = self.check_addr(param)
            val = self.memory[addr]
        elif mode == self.PARAM_MODE_REL:
            addr = self.relative_base + param
            addr = self.check_addr(addr)
            val = self.memory[addr]
        elif mode == self.PARAM_MODE_IMD:
            val = param
        else:
            raise Exception(f"Unkown paramater mode: {param}")

        return val
        
    def set_paramater(self, mode, data):
        param = self.memory[self.instruction_ptr]
        self.instruction_ptr += 1
        
        if mode == self.PARAM_MODE_POS:
            addr = self.check_addr(param)
            self.memory[addr] = data
        elif mode == self.PARAM_MODE_REL:
            addr = self.relative_base + param
            addr = self.check_addr(addr)
            self.memory[addr] = data
        elif mode == self.PARAM_MODE_IMD:
            raise Exception("Set paramater can't be in immediate mode")
        else:
            raise Exception(f"Unkown paramater mode: {param}")            
        
    def parse_opcode(self):
        mode_opcode_str = '{:>05}'.format(str(self.memory[self.instruction_ptr]))
        
        # Reverse of the first three chars
        modes = mode_opcode_str[:3][::-1]
        
        # integer of the last two chars
        opcode = int(mode_opcode_str[3:])
                     
        self.instruction_ptr += 1
                     
        return modes, opcode
        

    def run(self):
        self.output = None
        
        while self.instruction_ptr < len(self.memory):
            mode, opcode = self.parse_opcode()
            
            if opcode == self.OP_HALT:
                break
                
            elif opcode == self.OP_ADD:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                self.set_paramater(mode[2], in1 + in2)
            
            elif opcode == self.OP_MULTIPLY:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                self.set_paramater(mode[2], in1 * in2)
            
            elif opcode == self.OP_INPUT:
                if self.inputs:
                    self.last_input = self.inputs.pop()
                    
                if self.last_input != None:
                    self.set_paramater(mode[0], self.last_input)
                else:
                    raise Exception(f"{self.last_input} is not a valid input")
                
            elif opcode == self.OP_OUTPUT:
                self.output = self.get_paramater(mode[0])
                break # It took me way to long to realize I need to break here
                
            elif opcode == self.OP_JUMP_TRUE:
                do_jump = self.get_paramater(mode[0])
                new_addr = self.get_paramater(mode[1])
                if do_jump != 0:
                    self.instruction_ptr = new_addr
                
            elif opcode == self.OP_JUMP_FALSE:
                do_jump = self.get_paramater(mode[0])
                new_addr = self.get_paramater(mode[1])
                if do_jump == 0:
                    self.instruction_ptr = new_addr
                    
            elif opcode == self.OP_LESS_THAN:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                if in1 < in2:
                    self.set_paramater(mode[2], 1)
                else:
                    self.set_paramater(mode[2], 0)
                
            elif opcode == self.OP_EQUALS:
                in1 = self.get_paramater(mode[0])
                in2 = self.get_paramater(mode[1])
                if in1 == in2:
                    self.set_paramater(mode[2], 1)
                else:
                    self.set_paramater(mode[2], 0)
            
            elif opcode == self.OP_MOD_REL:
                val = self.get_paramater(mode[0])
                self.relative_base += val
                
            else:
                raise Exception(f'Unknown opcode {opcode} at addr {self.instruction_ptr}.')
                self.reset()
                
        return self.output

class TestBasic(unittest.TestCase):
            
    def test_rel_base(self):
        data = [([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99], 109),
                ([1102,34915192,34915192,7,4,7,99,0], 1219070632396864),
                ([104,1125899906842624,99], 1125899906842624)]
        for intcode, ans in data:
            iccp = IntcodeComputer(intcode)
            self.assertEqual(ans, iccp.run())
        
unittest.main(argv=[""], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [2]:
with open("inputs/input_d09.txt") as file:
    intcode = file.read()

iccp = IntcodeComputer(intcode)
iccp.add_input(1)
iccp.run()

3340912345

<h2 id="part2">--- Part Two ---</h2>
<p><em>You now have a complete Intcode computer.</em></p>
<p>Finally, you can lock on to the Ceres distress signal! You just need to boost your sensors using the BOOST program.</p>
<p>The program runs in sensor boost mode by providing the input instruction the value <code>2</code>. Once run, it will boost the sensors automatically, but it might take a few seconds to complete the operation on slower hardware. In sensor boost mode, the
    program will output a single value: <em>the coordinates of the distress signal</em>.</p>
<p>Run the BOOST program in sensor boost mode. <em>What are the coordinates of the distress signal?</em></p>

In [3]:
iccp.reset()
iccp.add_input(2)
iccp.run()

51754